In [1]:
import pandas as pd
import numpy as np 
import pylab as P 
# Import the module for plotting
import matplotlib.pyplot as plt
from pandas import Series,DataFrame,merge,concat
import pylab as P 
from datetime import datetime

In [2]:
# data import using the Data combined by Sectors
data = pd.read_csv('~/psychSignalData/output/DataBySectors.csv')

In [4]:
# data.columns
data.head()

,SECTOR,Sector_Index,SOURCE,SYMBOL,TIMESTAMP_UTC,BULLISH_INTENSITY,BEARISH_INTENSITY,BULL_MINUS_BEAR,BULL_SCORED_MESSAGES,BEAR_SCORED_MESSAGES,BULL_BEAR_MSG_RATIO,TOTAL_SCANNED_MESSAGES,TITLE,EXCHANGE,BUSINESS_SECTOR
0,Matrial,0,stocktwits+twitter_withretweets,AA,2009-08-24T04:00:00Z,1.19,0.0,1.19,1,0,0,2,Alcoa Inc.,NYSE,Material
1,Matrial,1,stocktwits+twitter_withretweets,AA,2009-09-03T04:00:00Z,1.33,0.0,1.33,1,0,0,2,Alcoa Inc.,NYSE,Material
2,Matrial,2,stocktwits+twitter_withretweets,AA,2009-09-10T04:00:00Z,2.50,2.3,0.20,1,1,1,2,Alcoa Inc.,NYSE,Material
3,Matrial,3,stocktwits+twitter_withretweets,AA,2009-09-11T04:00:00Z,2.40,0.0,2.40,1,0,0,2,Alcoa Inc.,NYSE,Material
4,Matrial,4,stocktwits+twitter_withretweets,AA,2009-09-15T04:00:00Z,0.00,0.0,0.00,0,0,0,2,Alcoa Inc.,NYSE,Material


# Compute the neutral scored message

In [5]:
neutralNum = data['TOTAL_SCANNED_MESSAGES']-data['BULL_SCORED_MESSAGES']-data['BEAR_SCORED_MESSAGES']

In [6]:
# insert number of neutral message counts
data.insert(11, 'NEUTRAL_SCORED_MESSAGES', neutralNum)
data.columns

Index(['SECTOR', 'Sector_Index', 'SOURCE', 'SYMBOL', 'TIMESTAMP_UTC',
       'BULLISH_INTENSITY', 'BEARISH_INTENSITY', 'BULL_MINUS_BEAR',
       'BULL_SCORED_MESSAGES', 'BEAR_SCORED_MESSAGES', 'BULL_BEAR_MSG_RATIO',
       'NEUTRAL_SCORED_MESSAGES', 'TOTAL_SCANNED_MESSAGES', 'TITLE',
       'EXCHANGE', 'BUSINESS_SECTOR'],
      dtype='object')

In [7]:
sentiNum = data['TOTAL_SCANNED_MESSAGES'] - data['NEUTRAL_SCORED_MESSAGES']

In [8]:
data.insert(12, 'TOTAL_SENTIMENT_MESSAGES',sentiNum)
data.columns

Index(['SECTOR', 'Sector_Index', 'SOURCE', 'SYMBOL', 'TIMESTAMP_UTC',
       'BULLISH_INTENSITY', 'BEARISH_INTENSITY', 'BULL_MINUS_BEAR',
       'BULL_SCORED_MESSAGES', 'BEAR_SCORED_MESSAGES', 'BULL_BEAR_MSG_RATIO',
       'NEUTRAL_SCORED_MESSAGES', 'TOTAL_SENTIMENT_MESSAGES',
       'TOTAL_SCANNED_MESSAGES', 'TITLE', 'EXCHANGE', 'BUSINESS_SECTOR'],
      dtype='object')

In [9]:
# remove useless columns
data = data.ix[:,3:]

# Select sub dataset 
- based on timestamp select tweets in 2014 and 2015

In [10]:
data['TIMESTAMP_UTC'] = data['TIMESTAMP_UTC'].apply(lambda x: datetime.strptime(x,'%Y-%m-%dT%H:%M:%SZ'))
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6025544 entries, 0 to 6025543
Data columns (total 14 columns):
SYMBOL                      object
TIMESTAMP_UTC               datetime64[ns]
BULLISH_INTENSITY           float64
BEARISH_INTENSITY           float64
BULL_MINUS_BEAR             float64
BULL_SCORED_MESSAGES        float64
BEAR_SCORED_MESSAGES        float64
BULL_BEAR_MSG_RATIO         float64
NEUTRAL_SCORED_MESSAGES     float64
TOTAL_SENTIMENT_MESSAGES    float64
TOTAL_SCANNED_MESSAGES      float64
TITLE                       object
EXCHANGE                    object
BUSINESS_SECTOR             object
dtypes: datetime64[ns](1), float64(9), object(4)
memory usage: 689.6+ MB


In [11]:
dateConstr = data[(data['TIMESTAMP_UTC'] > '2014-01-01') & (data['TIMESTAMP_UTC'] <= '2016-01-01')]

In [12]:
dateConstr.head()

,SYMBOL,TIMESTAMP_UTC,BULLISH_INTENSITY,BEARISH_INTENSITY,BULL_MINUS_BEAR,BULL_SCORED_MESSAGES,BEAR_SCORED_MESSAGES,BULL_BEAR_MSG_RATIO,NEUTRAL_SCORED_MESSAGES,TOTAL_SENTIMENT_MESSAGES,TOTAL_SCANNED_MESSAGES,TITLE,EXCHANGE,BUSINESS_SECTOR
1303,AA,2014-01-01 05:00:00,2.05,0.00,2.05,1,0,0.00,16,1,17,Alcoa Inc.,NYSE,Material
1304,AA,2014-01-02 05:00:00,1.90,1.80,0.10,18,1,18.00,51,19,70,Alcoa Inc.,NYSE,Material
1305,AA,2014-01-03 05:00:00,2.11,0.53,1.58,5,6,0.83,74,11,85,Alcoa Inc.,NYSE,Material
1306,AA,2014-01-04 05:00:00,2.28,0.00,2.28,5,0,0.00,27,5,32,Alcoa Inc.,NYSE,Material
1307,AA,2014-01-05 05:00:00,2.05,0.00,2.05,7,0,0.00,65,7,72,Alcoa Inc.,NYSE,Material


In [ ]:
# export the stock exchanges mereged dataset
dateConstr.to_csv('~/psychSignalData/output/selectedYear.csv', encoding='utf-8')

# select top 30 companies based on the total sentiment messages count

In [14]:
groupedCom = dateConstr.groupby(dateConstr['SYMBOL'])

In [17]:
sd = groupedCom.sum()

In [19]:
# sort data
sortedData = sd.sort('TOTAL_SENTIMENT_MESSAGES', ascending=False)
topCom = sortedData.head(30)

/Users/jessia/software/Anaconda/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


In [ ]:
topCom.to_csv('~/psychSignalData/output/top30compSum.csv', encoding='utf-8')